<a href="https://colab.research.google.com/github/rakshithcgowda/Feynman_symbolic_regression_MLH/blob/main/feynman_equations_by_symbolic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feymnam dataset regression by PHCRegressor

In original benchmark runs method with time budget 2 hours per one fit task. This notebook show that above 40% simplest Feynman equations can be solved in 1 second.

*(Original benchmark results are available here: https://cavalab.org/srbench/results/)*


## Installation

In [ ]:
%%capture
%pip install -U HROCH
#Penn Machine Learning Benchmarks
%pip install -U git+https://github.com/EpistasisLab/pmlb

In [ ]:
import numpy as np
import pandas as pd
import sympy as sp
import signal
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from collections import OrderedDict
from IPython.display import clear_output
from pmlb import fetch_data, regression_dataset_names
from HROCH import PHCRegressor

## Get Feynman equations

Read and parse original equations FeynmanEquations.csv and BonusEquations.csv (source: https://space.mit.edu/home/tegmark/aifeynman.html)


In [ ]:
eq_df = pd.read_csv('/kaggle/input/equations/FeynmanEquations.csv')
bonus_eq_df = pd.read_csv('/kaggle/input/bonus-equations/BonusEquations.csv')
eq_df.dropna(axis = 0, how = 'all', inplace = True)
bonus_eq_df.dropna(axis = 0, how = 'all', inplace = True)
eq_df.Filename = eq_df.Filename.apply(lambda x: 'feynman_' + x.replace('.', '_'))
bonus_eq_df.Filename = bonus_eq_df.Filename.apply(lambda x: 'feynman_' + x.replace('.', '_'))

eq_df = pd.concat([eq_df[['Filename', 'Formula']], bonus_eq_df[['Filename', 'Formula']]])

#feynman_I_15_10 in pmlb equal to I.15.1 in original source
eq_df.Filename = eq_df.Filename.apply(lambda x: x.replace('feynman_I_15_1', 'feynman_I_15_10'))
#replace arc functions for compatibility with sympy
eq_df.Formula = eq_df.Formula.apply(lambda x: x.replace('arcsin', 'asin'))
eq_df.Formula = eq_df.Formula.apply(lambda x: x.replace('arccos', 'acos'))
eq_df.Formula = eq_df.Formula.apply(lambda x: x.replace('arctan', 'atan'))
eq_df.head()

## Load datasets

In [ ]:
%%time
datasets={}
for name in regression_dataset_names:
    if name.find('feynman')>= 0:
        datasets[name] = fetch_data(name)


## Run experiment

In [ ]:
RANDOM_STATE = 42
SAMPLE_SIZE = 10000 #same as srbench
TIME_LIMIT = 1 #7200 in srbench, but we dont have 10 days to run this notebook
THREADS = 1 #same as srbench
TARGET_NOISE = 0 #0.001, 0.01, 0.1
SIMPLIFY_TIMEOUT = 1 #unfortunately sympy is sometimes too slow with simplification

In [ ]:
%%time

np.random.seed(RANDOM_STATE)
results = pd.DataFrame(columns=['dataset','equal','score', 'time', 'original', 'equation'])
cnt = 0
for name, df in datasets.items():
    cnt = cnt + 1
    #df = df.sample(n=SAMPLE_SIZE, random_state=RANDOM_STATE)
    X = df.drop('target', axis=1).to_numpy()
    y = df[['target']].to_numpy().ravel()
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=RANDOM_STATE)

    if len(y) > SAMPLE_SIZE:
        sample_idx = np.random.choice(np.arange(len(y_train)), size=SAMPLE_SIZE)
        X_train = X_train[sample_idx]
        y_train = y_train[sample_idx]
    if TARGET_NOISE > 0:
        y_train += np.random.normal(0, TARGET_NOISE*np.sqrt(np.mean(np.square(y_train))), size=len(y_train))

    reg = PHCRegressor(num_threads=THREADS, time_limit=TIME_LIMIT, random_state=RANDOM_STATE)

    start = time.time()
    reg.fit(X_train, y_train)
    fit_time = time.time() - start

    preds = reg.predict(X_test)
    score = r2_score(y_test, preds)

    orig = eq_df[eq_df.Filename == name]['Formula'].tolist()
    if (len(orig) == 0):
        print(f'equation {name} not found')

    results.loc[len(results)] = [name, None, score, fit_time, orig[0], reg.sexpr]
    clear_output()
    print(f'{cnt} {name} score: {score}')

clear_output()
results

## Postprocessing

* Compare with original equation.

In [ ]:
class TimeOutException(Exception):
    pass

def alarm_handler(signum, frame):
    print(f"raising TimeOutException")
    raise TimeOutException

signal.signal(signal.SIGALRM, alarm_handler)

def simplify(expr, **args):
    signal.signal(signal.SIGALRM, alarm_handler)
    signal.alarm(SIMPLIFY_TIMEOUT)
    try:
        expr2 = sp.simplify(expr, **args)
    except Exception as e:
        print('simplify ' + str(e))
        return expr
    signal.alarm(0)
    return expr2

def round_floats(ex1, precision=4):
    ex2 = ex1
    for a in sp.preorder_traversal(ex1):
        if isinstance(a, sp.Float):
            ex2 = ex2.subs(a, sp.Float(round(a, precision),precision))
    return ex2

def get_eq(X: pd.DataFrame, expr: str):
    features = [c for c in X.columns]

    eq = round_floats(sp.parse_expr(expr))
    model_str = str(simplify(eq, ratio=1))
    #mapping1 = {'x'+str(i+1): '$$$'+str(i+1) for i, k in enumerate(X.columns)}
    #mapping2 = {'$$$'+str(i+1): k for i, k in enumerate(X.columns)}
    mapping1 = OrderedDict({'x'+str(i+1): '$$$'+str(i+1) for i, k in enumerate(X.columns)})
    mapping2 = OrderedDict({'$$$'+str(i+1): k for i, k in enumerate(X.columns)})
    new_model = model_str
    for k, v in reversed(mapping1.items()):
        new_model = new_model.replace(k, v)
    for k, v in reversed(mapping2.items()):
        new_model = new_model.replace(k, v)
    return round_floats(sp.parse_expr(new_model, local_dict = {k:sp.Symbol(k) for k in features}))

def get_sym_model(X, expr, return_str=True):
    features = [c for c in X.columns if c != 'target']
    model_str = expr.replace('pi','3.1415926535')
    if return_str:
        return model_str

    model_sym = sp.parse_expr(model_str,local_dict = {k:sp.Symbol(k) for k in features})
    model_sym = round_floats(model_sym)
    return model_sym

def compare_sym_model(X: pd.DataFrame, expr1, expr2):
    model1 = expr1
    model2 = expr2

    try:
        sym_diff = round_floats(model1 - model2)
        sym_frac = round_floats(model1/model2)
        #print('sym_diff:',sym_diff)
        #print('sym_frac:',sym_frac)
        # check if we can skip simplification
        if not sym_diff.is_constant() or sym_frac.is_constant():
            sym_diff = round_floats(simplify(sym_diff, ratio=1))
            #print('simplified sym_diff:',sym_diff)

        symbolic_error_is_zero = str(sym_diff) == '0'
        symbolic_error_is_constant = sym_diff.is_constant()
        symbolic_fraction_is_constant = sym_frac.is_constant()
    except Exception as e:
        print(e)
        return False

    return symbolic_error_is_zero or symbolic_error_is_constant or symbolic_fraction_is_constant

In [ ]:
%%time
for index, row in results.iterrows():
    X = datasets[row["dataset"]].drop('target', axis=1)
    # skip inaccurate results
    if row['score'] < 0.99:
        results.at[index,'equation'] = str(get_eq(X, row['equation']))
        results.at[index,'equal'] = False
        continue

    eq = get_eq(X, row['equation'])
    orig_eq = get_sym_model(X, row['original'], False)

    equal = compare_sym_model(X, eq, orig_eq)
    results.at[index,'equation'] = eq
    results.at[index,'equal'] = equal

    clear_output()
    print(f'{index} {row["dataset"]} equal: {equal} [{eq}]')

clear_output()
results.head()

## Get and save results

* Get symbolic solution rate and accuracy (how often a method finds a model with test set R^2>0.999)

In [ ]:
solution_rate = results[results.equal == True].shape[0]/results.shape[0]
accuracy = results[results.score > 0.999].shape[0]/results.shape[0]

print(f'Symbolic solution rate:\t{round(solution_rate*100, 1)} %')
print(f'Accuracy:\t\t{round(accuracy*100, 1)} %')

* Save results

In [ ]:
results.to_csv('results.csv')

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(results.sort_values('score'))